In [1]:
import os

In [2]:
pwd


'c:\\Users\\Robin Aluma\\Desktop\\Car_Price_Pred\\Resources'

In [3]:
os.chdir('../')

In [4]:
pwd


'c:\\Users\\Robin Aluma\\Desktop\\Car_Price_Pred'

In [5]:

from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np
import sys
import pandas as pd

In [6]:

from src.Car_Price_Pred.utils.common import load_models,load_parameters,read_yaml,create_directories
from dataclasses import dataclass
from pathlib import Path
from src.Car_Price_Pred.constants import *
from sklearn.model_selection import train_test_split
from Exceptions import CustomException

In [7]:
@dataclass
class ModelEvaluationConfig():
    root_url: Path
    evaluation_data: Path
    best_model_path: Path
    decision_tree: str
    lasso: str
    linear_regression: str
    neighbors: str
    svm: str
    random_forest: str
    

In [8]:
class ModelEvaluationConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH):
        self.config = read_yaml(config_file_path)
        
        create_directories([self.config.Artifacts_root])
        
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_url])
        model_evaluation_config = ModelEvaluationConfig(
            root_url = config.root_url,
            evaluation_data = config.evaluation_data,
            best_model_path = config.best_model_path,
            decision_tree = config.decision_tree,
            lasso = config.lasso,
            linear_regression = config.linear_regression,
            neighbors = config.neighbors,
            svm = config.svm,
            random_forest = config.random_forest
        )
        
        return model_evaluation_config
    

In [9]:
class ModelEvaluation():
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config
        
    @property
    def get_evaluation_data(self):
        test_data = pd.read_csv(self.config.evaluation_data)
        x = test_data.iloc[:,:-1]
        y = test_data.iloc[:,-1]
        x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)
        
        
    @property
    def evaluate_model(self):
        x_train = self.x_train
        y_test = self.y_test
        x_test = self.x_text
        
      # Then use them in evaluation
    
        
         
        models = [
            load_models(self.config.decission_tree),
            load_models(self.config.lasso),
            load_models(self.config.linear_regression),
            load_models(self.config.neigbors),
            load_models(self.config.svm),
            load_models(self.config.random_forest)]
        
        for model in models:
            y_predict = model.predict(x_test)
            y_true = y_test
        
            return y_predict, y_true
        
    def calculate_metrics(self,y_true, y_pred):
        self.y_true = evaluation_model(y_true)
        self.y_pred = evaluate_models(y_pred)
        
        r_score = r2_score(self.y_true, self.y_pred)
        mae = mean_absolute_error(self.y_true, self.y_pred)
        mse = mean_squared_error(self.y_true, self.y_pred)
        rmse = np.sqrt(mse)
        
        return r_score,mae,mse,rmse

In [10]:
class ModelEvaluationPipeline:
    def __init__(self):
        pass
    def main(self):
        try:
            
            config = ModelEvaluationConfigurationManager()
            get_model_evaluation_config = config.get_model_evaluation_config()
            model_evaluation = ModelEvaluation(config = get_model_evaluation_config)
        
            # First store the evaluation data
            evaluation_data = model_evaluation.get_evaluation_data()
        
            # Then use the stored data for evaluation
            model_evaluation.evaluate_model()
            model_evaluation.calculate_metric()
        except Exception as e:
            
            raise CustomException(e,sys)
        

In [11]:
if __name__ == '__main__':
    try:
        obj = ModelEvaluationPipeline()
        obj.main()
    except Exception as e:
        raise CustomException(e,sys)

[ 2025-01-21 06:04:26,982 : common : INFO : Creates directory for path;artifacts in file paths :['artifacts'] ]
[ 2025-01-21 06:04:26,983 : common : INFO : Creates directory for path;artifacts/model_evaluation in file paths :['artifacts/model_evaluation'] ]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Robin Aluma\AppData\Local\Temp\ipykernel_13152\3695319764.py", line 12, in main
    evaluation_data = model_evaluation.get_evaluation_data()
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: 'NoneType' object is not callable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Robin Aluma\AppData\Local\Temp\ipykernel_13152\2051446063.py", line 4, in <module>
    obj.main()
  File "C:\Users\Robin Aluma\AppData\Local\Temp\ipykernel_13152\3695319764.py", line 19, in main
    raise CustomException(e,sys)
Exceptions.CustomException: Error occured in python script no ['NoneType' object is not callable] line number [12] error message [C:\Users\Robin Aluma\AppData\Local\Temp\ipykernel_13152\3695319764.py]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Robin Aluma\AppData\Roaming\